Neural Model Identification: \
Trains a neural network based on data from the physics model to use as the predictive model
Note: You need to install the matlab engine to be able to run matlab code from pythone

In [210]:
import matplotlib.pyplot as plt

In [211]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#from skopt.space import Real, Categorical, Integer
from sklearn import preprocessing

In [212]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [213]:
import scipy.io
model_data = scipy.io.loadmat('Data.mat')

In [214]:
Zin=model_data.get("Zin")
Zout=model_data.get("Zout")

In [215]:
def swish(x, beta=1):
    return x * sigmoid(beta * x)

In [252]:
def get_model_layers(nodes, layers, activation, learning_rate, training=None):
    if activation == 'swish':
        activation = swish
    else:
        activation = activation

    inputs = tf.keras.layers.Input(shape=(in_dim,))
  #  xadd = (np.array(inputs[:, :out_dim])).reshape(None,1)
   # print(inputs[:, :out_dim].shape)
    #print(xadd.shape)
    x = tf.keras.layers.Dense(units=nodes, activation=activation)(inputs)
    for i in range(layers - 1):  # at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation=activation)(x)

    #x = tf.keras.layers.Dropout(rate=0.1)(x, training=training)
    outputs = tf.keras.layers.Dense(out_dim, activation="linear")(x)
   # outputs = tf.keras.layers.Add()([outputs, inputs[:, 2:]])

    #outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]])
    # outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]]) NN4UQ
    #    out = keras.layers.Dense(units=3, activation= "linear")(added)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error')
    return model

In [253]:
get_custom_objects().update({'swish': Activation(swish)})

In [254]:
X_train, X_test, Y_train, Y_test = train_test_split(Zin, Zout, shuffle=True, train_size=0.8, random_state=42)
in_dim = X_train.shape[1]
out_dim = Y_train.shape[1]

In [257]:
learning_rate=0.05#gp_result.x[0]
num_layers = 3 #gp_result.x[1]
num_nodes = 5 #gp_result.x[2]
batch_size=10 #gp_result.x[3]
epochs=300
activation='swish'
DropModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                             activation=activation, learning_rate=learning_rate,
                             training=True)  # True-activates dropout during prediction
DropModel.summary()
DropModel.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data = (X_test, Y_test))

DetModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                            activation=activation, learning_rate=learning_rate)
#name = '{}_{}_{}_{}'.format(num_nodes, num_layers, activation, learning_rate)
#DropModel.save(root + 'results/nc/' + name)
DetModel.set_weights(DropModel.get_weights())
score = DetModel.evaluate(X_test, Y_test, verbose=0)

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_64 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_66 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_67 (Dense)             (None, 3)                 18        
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________
Train on 1600 samples, validate on 400 samples
Epoch 1/300
1600/1600 [==============================] - 1s 427us/sample - loss

Epoch 68/300
1600/1600 [==============================] - 0s 96us/sample - loss: 0.0017 - val_loss: 0.0016
Epoch 69/300
1600/1600 [==============================] - 0s 94us/sample - loss: 0.0017 - val_loss: 0.0019
Epoch 70/300
1600/1600 [==============================] - 0s 90us/sample - loss: 0.0017 - val_loss: 0.0013
Epoch 71/300
1600/1600 [==============================] - 0s 90us/sample - loss: 0.0017 - val_loss: 0.0013
Epoch 72/300
1600/1600 [==============================] - 0s 85us/sample - loss: 0.0016 - val_loss: 0.0020
Epoch 73/300
1600/1600 [==============================] - 0s 82us/sample - loss: 0.0016 - val_loss: 0.0014
Epoch 74/300
1600/1600 [==============================] - 0s 89us/sample - loss: 0.0016 - val_loss: 0.0017
Epoch 75/300
1600/1600 [==============================] - 0s 89us/sample - loss: 0.0016 - val_loss: 0.0015
Epoch 76/300
1600/1600 [==============================] - 0s 87us/sample - loss: 0.0016 - val_loss: 0.0016
Epoch 77/300
1600/1600 [=============

1600/1600 [==============================] - 0s 94us/sample - loss: 0.0012 - val_loss: 0.0012
Epoch 145/300
1600/1600 [==============================] - 0s 93us/sample - loss: 0.0012 - val_loss: 9.7023e-04
Epoch 146/300
1600/1600 [==============================] - 0s 92us/sample - loss: 0.0012 - val_loss: 0.0012
Epoch 147/300
1600/1600 [==============================] - 0s 86us/sample - loss: 0.0012 - val_loss: 0.0010
Epoch 148/300
1600/1600 [==============================] - 0s 92us/sample - loss: 0.0012 - val_loss: 0.0015
Epoch 149/300
1600/1600 [==============================] - 0s 87us/sample - loss: 0.0012 - val_loss: 0.0012
Epoch 150/300
1600/1600 [==============================] - 0s 88us/sample - loss: 0.0012 - val_loss: 0.0012
Epoch 151/300
1600/1600 [==============================] - 0s 90us/sample - loss: 0.0012 - val_loss: 0.0012
Epoch 152/300
1600/1600 [==============================] - 0s 90us/sample - loss: 0.0012 - val_loss: 0.0014
Epoch 153/300
1600/1600 [=============

1600/1600 [==============================] - 0s 79us/sample - loss: 0.0011 - val_loss: 0.0011
Epoch 220/300
1600/1600 [==============================] - 0s 82us/sample - loss: 0.0011 - val_loss: 0.0011
Epoch 221/300
1600/1600 [==============================] - 0s 86us/sample - loss: 0.0011 - val_loss: 9.1060e-04
Epoch 222/300
1600/1600 [==============================] - 0s 78us/sample - loss: 0.0011 - val_loss: 0.0013
Epoch 223/300
1600/1600 [==============================] - 0s 79us/sample - loss: 0.0011 - val_loss: 0.0012
Epoch 224/300
1600/1600 [==============================] - 0s 81us/sample - loss: 0.0011 - val_loss: 0.0010
Epoch 225/300
1600/1600 [==============================] - 0s 81us/sample - loss: 0.0011 - val_loss: 0.0011
Epoch 226/300
1600/1600 [==============================] - 0s 84us/sample - loss: 0.0011 - val_loss: 9.7713e-04
Epoch 227/300
1600/1600 [==============================] - 0s 89us/sample - loss: 0.0011 - val_loss: 9.3065e-04
Epoch 228/300
1600/1600 [=====

1600/1600 [==============================] - 0s 82us/sample - loss: 0.0010 - val_loss: 9.7622e-04
Epoch 294/300
1600/1600 [==============================] - 0s 85us/sample - loss: 0.0010 - val_loss: 0.0012
Epoch 295/300
1600/1600 [==============================] - 0s 83us/sample - loss: 0.0010 - val_loss: 0.0010
Epoch 296/300
1600/1600 [==============================] - 0s 80us/sample - loss: 0.0010 - val_loss: 0.0011
Epoch 297/300
1600/1600 [==============================] - 0s 87us/sample - loss: 0.0010 - val_loss: 9.2859e-04
Epoch 298/300
1600/1600 [==============================] - 0s 85us/sample - loss: 0.0010 - val_loss: 8.8040e-04
Epoch 299/300
1600/1600 [==============================] - 0s 82us/sample - loss: 0.0010 - val_loss: 9.7853e-04
Epoch 300/300
1600/1600 [==============================] - 0s 81us/sample - loss: 0.0010 - val_loss: 0.0011


In [258]:
weights = DropModel.get_weights() # returs a numpy list of weights

In [259]:
Wmat = np.array(weights)

/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [260]:
def predictive_model_resnet(zk,Wmat): 
    #Assuming you can pass extra arguments to the function for casadi, otherwsise
    #Number of hidden layers and nodes are given once when we get the optimal resnet
    #The Bayesian Optimization Loop will be changing 
    n_hidden_layers = 3
    n_nodes = 5
    model_input = zk.reshape(1,5)
    Wmat_shape = Wmat.shape[0]
    
    #Initialize from input 
    sum0 = np.matmul(zk,Wmat[0]) #Matrix multiplication weights*inputs
    sumbias0 = sum0 + Wmat[1] #Add bias to construct y = sum(wi*xi) + b
    #neuron_out = swish_eval(sum1bias)
    neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation, out = fact(y)
    #They are vector quantitites so you might need to change stuff a bit with symbolic vars
    print(neuron_out.shape)

    #Forward prediction in hidden layers
    for k in range(n_hidden_layers-1):
        sum0 = np.matmul(neuron_out,Wmat[2*k+2])
        sumbias0 = sum0 + Wmat[2*k+3]
        neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation

    #Final output layer with linear activation
    sum0 = np.matmul(neuron_out,Wmat[Wmat_shape-2])
    sumbias0 = sum0 + Wmat[Wmat_shape-1]
    neuron_out = sumbias0  #linear activation

    return neuron_out #+ model_input[0,2:5]

In [261]:
zk=Zin[0,:].reshape(1,5)

In [262]:
zkplusone  = predictive_model_resnet(zk,Wmat)

(1, 5)


In [266]:
zk[0,0]

0.11050563422067754

In [267]:
print('Current State:', zk,'Future State', zkplusone)

Current State: [[0.11050563 0.6529952  0.3        0.         0.        ]] Future State [[ 3.00609509e-01 -8.44025685e-05 -4.99411929e-04]]


In [268]:
initial_cond_plant =scipy.io.loadmat('init_cond.mat')

In [269]:
xo = initial_cond_plant.get("yo")

In [270]:
xo.shape

(1, 87)

In [271]:
x0 = xo[0,-3:]

For BCR_Discrete you must convert the states array to a list like below and then put it into matlab.double; \
Same for matlab.double for the scalars \
The other, fixed parameters are ok like that

In [272]:
zkplusone_plant = eng.BCR_Discrete(matlab.double(xo.tolist()),matlab.double([zk[0,0]]),matlab.double([zk[0,1]]),82.3,1.0,0.9)

Convert zkplusone_plant to np.array since it's a matlab type initially 

In [273]:
zkfuture_plant = np.array(zkplusone_plant)

In [274]:
zkfuture_plant[0,-3:] #Getting the last states

array([0.29794952, 0.02033513, 0.0067755 ])

In [275]:
zkfuture_plant  = predictive_model_resnet(np.array([zk[0,0],zk[0,1],x0[0],x0[1],x0[2]]),Wmat)

(5,)


In [276]:
zkfuture_plant

array([ 3.00609509e-01, -8.44025685e-05, -4.99411929e-04])

One of the issues here with the zkfuture_plant

When you are at the initial condition, or acetate and ethanol are very low, usually you will predict negative concentrations for acetate and ethanol. In other words, you will predict x[1], and x[2] <0 which is unphysical. In this case you can make the prediction equal to zero (because otherwise you might not get feasible solutions)

#Objective Function
Area = 0.002436;   % reactor cross-sectional area, m^2 \
L = 1.06;   % reactor length, m  \
Qmedia[j] = Area \times L \times Dk[j]  (Dk is u[0], i.e., dilution rate) \
Try to maximize sum (Qmedia[j]*x1[j] ) (m3/h * mmol/m^3)  \
Constraints:
Selectivity is defined as Ce/Ca or x2/x1 and I want it to be x2/x1 < 0.45 \
u[0] in (0.01, 0.25) \
u[1] in (0.0, 0.9) \ 
x0,x1,x2 > 0 
